## RECOMMENDER SYSTEM FOR E-COMMERCE

## MODELLING: Part 2

in the first part of the modelling we have used simple algorithms to develop 2 collaborative filtering models: KNN and Matrix Factorization
We have seen that the Matrix Factorization performance was more stable, however, our RMSE score over a sample of 1000 interactions was 15 which is not very great

The objective of this notebook is to use the scikit learn surprise library to explore alternative algorithms that can perform better.
Also here, we will evaluate precision and recall as addiitional metrics 

### Imports

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise import KNNBasic, SVD, Dataset, NormalPredictor, Reader, accuracy

In [2]:
data = pd.read_csv('/Users/judith/Data_science_projects/Springboard_AssignmentsJY/capstone_three/data/processed/processed_ratings_data.csv')

In [3]:
data.head()

,item_id,user_id,rating
0,7443,Alex,4
1,7443,carolyn.agan,3
2,7443,Robyn,4
3,7443,De,4
4,7443,tasha,4


In [4]:
reader = Reader()
df = Dataset.load_from_df(data[['user_id', 'item_id', 'rating']], reader)

In [7]:
# Finding the best model
benchmark = []
for algo in [SVD(), KNNBasic(), NormalPredictor()]:
    results = cross_validate(algo, df, measures = ['RMSE'], cv = 5, verbose = False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]],index=['Algo']))
    benchmark.append(tmp)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [9]:
surprise_results = pd.DataFrame(benchmark).set_index('Algo').sort_values('test_rmse')
surprise_results

,test_rmse,fit_time,test_time
Algo,,,
SVD,1.056060,6.689027,0.222243
KNNBasic,1.080752,191.928903,11.370964
NormalPredictor,1.393228,0.234222,0.372735


In [11]:
# Using grid search to find the best parameters for our best model
param_grid = {'n_factors': [10, 15, 20],
             'n_epochs': [5,10,15],
             'lr_all': [0.003, 0.005]}

grid_search = GridSearchCV(SVD, param_grid, measures = ['rmse'], cv = 5)
grid_search.fit(df)
algo = grid_search.best_estimator['rmse']
print(grid_search.best_score['rmse'])
print(grid_search.best_params['rmse'])

1.0426064203882714
{'n_factors': 10, 'n_epochs': 10, 'lr_all': 0.005}


In [ ]:
# Extracting parameters

In [14]:
factors = grid_search.best_params['rmse']['n_factors']
epochs = grid_search.best_params['rmse']['n_epochs']
lr_value = grid_search.best_params['rmse']['lr_all']

## Train and Test Split

In [15]:
train, test = train_test_split(df, test_size = 0.2)
best_model = SVD(n_factors = factors, n_epochs = epochs, lr_all = lr_value)
best_model.fit(train)
predictions = best_model.test(test)

In [19]:
accuracy.rmse(predictions, verbose = True)

RMSE: 1.0422


1.0421512290051052

In [20]:
predictions

[Prediction(uid='Melody', iid=58940, r_ui=5.0, est=4.378604826837305, details={'was_impossible': False}),
 Prediction(uid='Vicki', iid=62482, r_ui=5.0, est=4.291739586557089, details={'was_impossible': False}),
 Prediction(uid='Ahartman', iid=34937, r_ui=5.0, est=4.524685207757566, details={'was_impossible': False}),
 Prediction(uid='elizabeth.mcgovern', iid=110004, r_ui=4.0, est=4.015964744539428, details={'was_impossible': False}),
 Prediction(uid='mitiq15', iid=77878, r_ui=4.0, est=4.3861014710371835, details={'was_impossible': False}),
 Prediction(uid='timika', iid=152111, r_ui=1.0, est=4.1487922873109495, details={'was_impossible': False}),
 Prediction(uid='TheBlueSwan', iid=82256, r_ui=5.0, est=4.594246655813035, details={'was_impossible': False}),
 Prediction(uid='sollasollew7', iid=113512, r_ui=3.0, est=4.4952383445163315, details={'was_impossible': False}),
 Prediction(uid='ysmithco', iid=57369, r_ui=4.0, est=3.7187447486948884, details={'was_impossible': False}),
 Prediction(

In [50]:
# Defining a function to calculate precision and recall

def precision_recall_at_k(predictions, k, threshold):
    user_est_true = defaultdict(list)
    for user_id, _,rating, est_rating, _ in predictions:
        user_est_true[user_id].append((est_rating, rating))
    precision = dict()
    recall = dict()
    for user_id, user_rating in user_est_true.items():
        user_rating.sort(key = lambda x: x[0], reverse = True)
        # nber of relevant items
        relevant = sum((rating >= threshold) for (_, rating) in user_rating)
        # number of recommended items in top k
        recommended = sum((est_rating >= threshold) for (est_rating, _) in user_rating[:k])
        # number of releant and recommended items in top k
        rel_recom =sum(((rating >= threshold) and (est_rating >= threshold)) for (est_rating, rating) in user_rating[:k])
            #precision
        precision[user_id] = rel_recom/recommended if recommended != 0 else 1
            
            # recall
        recall[user_id] = rel_recom/relevant if relevant != 0 else 1
    return precision, recall

In [51]:
# Getting results
thresholds = np.arange(start = 2,stop = 5, step = 1)
results = []
for k in range(5,11):
    for threshold in thresholds:
        precision, recall = precision_recall_at_k(predictions, k, threshold)
        pre = sum(prec for prec in precision.values())/ len(precision)
        rec = sum(rec for rec in recall.values()) / len(recall)
    results.append({'K':k, 'threshold':threshold, 'precision': pre, 'recall': rec})
results

[{'K': 5,
  'threshold': 4,
  'precision': 0.8613908964118172,
  'recall': 0.8455353672891684},
 {'K': 6,
  'threshold': 4,
  'precision': 0.8611271712945354,
  'recall': 0.8473268817190241},
 {'K': 7,
  'threshold': 4,
  'precision': 0.8609677769928815,
  'recall': 0.8485294392107526},
 {'K': 8,
  'threshold': 4,
  'precision': 0.8608672499049069,
  'recall': 0.8494112519598008},
 {'K': 9,
  'threshold': 4,
  'precision': 0.8607425721651665,
  'recall': 0.8500166061393133},
 {'K': 10,
  'threshold': 4,
  'precision': 0.8606757956130343,
  'recall': 0.8505158030026391}]

In [44]:
# We will build a dataframe to compare the results
results_df = pd.DataFrame(results).sort_values(by = 'precision')
results_df

,K,threshold,precision,recall
5,10,4,0.860676,0.850516
4,9,4,0.860743,0.850017
3,8,4,0.860867,0.849411
2,7,4,0.860968,0.848529
1,6,4,0.861127,0.847327
0,5,4,0.861391,0.845535


### Making recommendations

In [45]:
# This function fetch recommendations from predictions
n = 10
def make_recommendations(predictions, n):
    top_n = defaultdict(list)
    # map predictions to each user
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        # sort prediction to return the top n ones
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [48]:
# Applying the recommendation function of a sample of 10 items
top_n = make_recommendations(predictions, 10)

In [49]:
for iud, user_ratings in top_n.items():
    print(iud, [iid for (iid, _) in user_ratings])

Melody [58940]
Vicki [69157, 50405, 62482, 105593, 138225, 77959]
Ahartman [34937]
elizabeth.mcgovern [110004]
mitiq15 [77878]
timika [152111]
TheBlueSwan [82256]
sollasollew7 [113512, 84931]
ysmithco [57369]
inkywinks02 [77998, 21296]
leesarbelle [82288]
Tara [153711, 70478, 34935, 126993, 126114, 62482, 68332, 31644, 114610, 55967]
Haley [40899, 104356, 118020, 106758, 110842, 60724, 62447, 107041]
yerma [142524]
elizaschwarm [16411]
Kathi [139488, 106758, 146475, 128359]
schume89 [68332]
ameejoshi [113512]
kristinejditmarson [146840]
carly [153473, 113512, 32406, 107041, 150698]
torikill [82288]
katy32 [32236]
Adryan [153598, 153599]
mud729 [84928]
Emma [133539, 59176, 28252, 32406, 107794, 113510, 22563, 47397, 131290, 84436]
Erica [69157, 152793, 103764, 34931, 82256, 141255, 137285, 68332, 153517, 75428]
Lindsay [70538, 106758, 32236, 104741, 31644, 113643, 50818, 80238, 71607, 154334]
Taylor [67022, 114010, 34935, 145313, 32405, 153571, 141845, 47657, 137285, 55967]
Tallulah [72

Violet [153222]
Crystal [35272, 129113, 49525, 68332, 55967]
michelle [67022, 114770, 82288, 28252, 70538, 143049, 135364, 153721, 151842, 107539]
raspberry.oneill [89042]
Alexandra [34935, 102736, 89044, 150509, 49525, 153961, 113643, 120217, 31644, 107794]
virginia [140727, 107794, 118948, 31752, 77959]
nekocalie [82092]
aurora.cooper [128406]
Kirby [34937]
sarah.e.miller87 [67507]
Littlemissallee [153373]
Vasilisa [129096]
havefaith92 [54887]
bridget.a.schwartz [112771]
brittany [59176, 143434, 69630, 151966, 22563]
klars622 [110004]
hillary.skaer92 [114610]
ashleestanley [54222]
Rabbit [124599]
monsterlover88 [69157]
ahealey777 [77879]
kelleymcnabbdunham [82288]
kat.carvell [128405]
jodi.beyer [58940]
Louise [146840, 64921, 50405, 70538, 152426, 109327, 47397, 154139, 105202, 81802]
hellojackie1105 [55967]
Gretchen [70478, 153462, 82256, 153439, 110824, 36172, 31752]
lmohhhh [59176]
Margaret [76049, 112559, 69157, 32236, 153481, 154568, 112506, 128406, 7443]
Mel [153709, 72816, 849

skoutabout78 [114371, 28967]
ladyliz78 [69157, 145278]
ella [34935]
meeyanda [129278]
taniafairchild [47657]
meowyloumew [84931]
jenybean [21296]
jlmhall [153167]
Jinx [80427, 114610]
telma [31752]
Nataliya [110820]
timmerman.jen [153370, 62482]
teresa [40899, 67194, 80841, 89044, 11960, 154216, 153847]
lil_chi [65046]
lyndseycheyenne96 [141856]
shannonsodonnell [67194, 32403, 57369]
velocitysc43 [78573]
jupretzer [35272, 115417]
amparo [64745, 67397]
jossa_86 [40899, 54222, 73186]
Dakota [67531, 55967, 76481, 110004]
kate [35272, 40899, 34935, 133399, 80841, 54222, 77951, 152052, 153295, 85507]
mowaltrod [126114]
Maribeth [110842, 107041]
erika [112559, 77949]
Francesca [128406, 57369]
Dawn [76049, 59176, 40899, 34935, 32236, 34931, 32403, 114610, 11960, 71434]
dani.jenkins [64921]
Christy [34935, 32403, 78573, 32406, 153721, 148245, 105202, 110004, 57369]
Sam71 [133300]
camilleeeon [11960]
tamminic7 [44261]
schomburg [150788]
charity [137370, 40899, 154665]
eileenmcg523 [31644]
julia

mhartrp [54222]
ATX [129267]
goldenmesa [32405, 113510, 57369]
kim [106758, 32403, 137270, 68332, 36172, 143422]
christinestephens1 [22563, 24853]
shannonltierney [28967, 67194, 49587]
tippipatrick [111984]
emileeljohnson [77951, 67197]
nicholson.h [16411]
rb1996chica [32406]
AlishaG [32406]
pamelajameswla [32406]
j.carberry233 [67507]
remington.sarah [70538]
ippy5 [68332]
charrotoby [143347]
emi.jones [32406]
Dell [82288]
bunkaboo338 [102736]
meaghan.o [84436]
annettedesigns [49525]
jsiebenmorgen [11960]
jlmccrim [32405]
Alexi [150500]
corissa_depeel [116716]
loglloyd [28252]
msmo423 [103764]
Marie-Pier [150864]
meleaton [141110]
TheLadyChar [153715]
jcnq3 [143872]
christymarie1055 [50818]
tomcar03 [34935, 34937]
mxfranz [62447]
maisie [131290]
lynna2345 [135362]
tropicalstorm83 [67531]
dinahbmiller [77879]
mlblack09 [106758]
Jackie [67022, 113512, 70538, 49525, 121272, 77878, 24853, 152616]
priyanka [84928]
amanda.t.parrish [34931]
customer15197971 [80427]
alysia [54222]
artsychick42

Nina [32403, 107539, 77879, 65046, 16411]
maggietracy [110004]
Buttercup [141856, 153108]
shannimals [107043]
cmynoggin [34935]
jackie.degraf [122266]
jdkring74 [84436]
samantha.s.stockwell [34937]
Meowster [59176, 32403, 65046]
kac1022 [80238]
honeydew8329 [80841]
Jasmyn [55967]
brandy.walls [107043]
TAYLOR [110004]
sberndt1504 [24853]
geraniumkisser [22563]
amberbemis [76049, 28252, 67194, 105593, 146835]
styteller2 [27439]
primepasturedpeeps [60724]
oneiricbeing [61110]
kt_a_11 [68332]
Jean [32405, 7443, 16411]
Angelique [82288]
hogg_s [82288, 28967, 56780]
mirandahuffman [34935]
RainyM [44261]
darcy [114770, 151870, 129278]
Kiko [124599]
SARAH [76049, 56780]
katie.beall6 [34935]
elofgren20 [114609]
Peggy [44893]
jmayr [59176]
Kikicat [154266]
papia.nandi [16411]
yvette [113512, 152448]
bennyanddoone [52406]
tas90987 [152891]
Bahar [27439]
Kendall [153393, 76049, 80427, 114609, 153961, 58940, 153867]
cristin.arnett [110842, 7443]
misshanie [77878]
ruth [153303, 153606, 128359]
krist

...

chloe_bessette [64745]
caitlin.bales [32406]
arachelk [28967]
elizabethrstark [40899]
amandajune11 [82256]
f. [32406]
valerie [21296, 120563, 89042]
nmfonteyne [110820]
corylee [27590]
demojapan1 [67022]
Madi [80841]
sdunbar12 [11960]
jeostbye [57369]
miega_loba [78573]
lanternowl [67022]
shanellelewis5 [123653]
cecidadisman [16411]
jes.lyon0 [110820]
slgoff87 [69630]
lorbug90 [57369]
oogloriaaa [50818]
kate.wegener88 [62447]
anna_rowland [153373]
acrossley [28252]
Lost [57369]
bayleybryan85 [140641]
Mikaela [152696, 72816]
Priti [150849]
laurenkp [71434]
jesssilv26 [116736]
librarinista [11960]
blu3idb3auty [137370, 55967]
vlsquires [87051]
grabower [48629]
tmgross2 [80841]
sreya [34935]
daniellekellogg [24853]
o_o [55967]
Val14v [129160]
Sandy [82288, 71434]
lenoremaier [76049]
samanthasynold [107043]
jcpollutra [16411]
davis.katev [71607]
kerri.swails [80003]
kathleen.mettle [124599]
kels.mc [129278]
cpoh [59176, 49587]
carrie416 [31752]
iris [129113]
cindy_bacon [48629]
classydame 

hilaryjean [128405]
Taryn [152035]
mmmjulie [77625]
frankie [153462]
diane.davis5588 [106758]
catherinekastens [70538]
CC [75947]
carolransley [135714]
Jaimes [150866]
Poetka [103764]
grace_johnstone75 [31752]
isabeltorres2006 [82496]
xxx [31752]
crouchsarah [62447]
calehto [85507]
morales.evelyn15 [135364]
h.rushing [34931]
sylviamaynewton [70538]
lizz91 [82496]
joosthuizen [153393]
Sangdi [16411]
kristi.cheshire [140192]
jromerops1 [65046]
knorgaard31 [80841]
roseuke [50818]
i [28967]
October [84931]
intothewild.ab [78224]
yelhsa9991 [32406]
Jessiqua [47657]
b.felicia.acosta [67531]
sydney.rooney [54222]
csalley88 [32403]
henkin.kelila [76049]
lbthomas090 [32405]
mclaire [34931]
Carisa [62447]
bderwis [16411]
roselaine.fox [34935]
camccullough [28252]
erin.stuhlman [84436]
most.amazing.email [114770]
dezzviv [116736]
nursechrissy09 [152929]
miVi3k [28252]
sarahaharrison92 [120558]
shel_tak [32405]
smrusk [117620, 49587]
antionette_miller20 [116736]
lrproloson [31644]
rem02786 [107043

kristina85marie [44893]
steph [107539]
BaltBombshell [153452]
treesgetwheeledaway [131290]
jessicaherran [68332]
divya [145313]
RACHEL [24853]
dogwithbooks [143764]
shag0717 [125077]
niemi.claire [59176]
kela.binion [27439]
wishmaker53 [80238]
libbynatola19 [84436]
ibonilla7 [73186]
dramatichippie [40899]
hollers25 [82288]
laurenjsiegert [32405]
rhonna [153613]
jkwozney [82288]
hgeorgia27 [34937]
andreya [147870]
msprot87 [67507]
yetified21 [110824]
everythingisstardust [27439]
hbbreeze17 [57369]
kgeney [78510]
vadame [82256, 47397]
shadowsmom10 [62482]
johanna [153438]
reinkehannah [67507]
howhow2 [22563]
sabrandejs [126098]
ma92845 [31644]
cjsitko [32236]
katherineeconaty [114770]
RaLaFo. [154262]
Lafayette [107539]
Juana [21296]
omorales [82496]
ariessumbad [32406]
Leila [153184]
calli_w [57369]
shcd6154 [36172]
scruggsj [89044]
sarah.white [34935]
Pony [153374]
kpennington101 [31644]
Bem [153386]
jnelson2386 [89042]
boiga123 [111984]
twingeofirony [102736]
elizabethafrancisco [2485

sarahcrevell [28967]
franchesca.judd [67507]
anc0289.bsn [103764]
loves2laughaq [123500]
madilowe [144759]
smorris8 [82288]
charlotte.doucette89 [54222]
kd7105 [114609]
Riley [72816, 73186]
meyasigi [103764]
woman [50405]
emvav1993 [141025]
careydragone [77625]
DJhonna [21296]
katherine.gardner44 [76049]
dauwkn [50818]
jayneebabe [69157]
oclark12 [70538]
jomoore1980 [77949]
impsythealmighty [120558]
moulton.j [21296]
angiechrisman79 [84436]
Cate [110004]
viki.barpalias [32403]
amanda.m.vierling [106758]
inariwarrior [28967]
leahbee86 [56289]
mistels [27439]
panifor [35525]
poisongreen [67397]
lillart [62447]
Corrine [82256]
lady_lovely_locks1 [137816]
tdemink [68332]
zaida [152020]
smithcn [72816]
dfmondello [57369]
elonda5 [122266]
gabby24382 [105593]
e.baarman [78573]
Crystle [34935]
firedrake42 [151095]
leahmorast [84928]
katie_winstanley [103764]
erindkerrigan [137371]
Miss_Chams [59176]
tahlia.jovic [55967]
utbabe1226 [114770]
a.michelle19 [67194]
Jodytron [151263]
glitterhalos [2

kcw760 [140192]
libby.cowgill [103764]
animeflower [21296]
acj512 [75947]
mitzymetzy [126114]
kbrown1516 [48629]
sugarjess1118 [32403]
millie.elizabeth.almond [141845]
dorry_1 [21296]
ellmo530 [114610]
pongodal1 [146835]
fabulousboo90 [21296]
CJM7 [86074]
tricedan [77878]
holly.harwood [128563]
Cressy [131290]
Shelise [49587]
sgeorge_9 [57369]
brittanylynn17 [82288]
peitabonato [128563]
rosencrayons [107539]
jenncatt3 [16411]
george.betsym [153434]
jami24chung [104741]
heidi.hartshorn [28967]
Amy94 [144619]
charrodie [106242]
RittleBerri [28967]
Godzilla [141295]
blmclain526 [67022]
brndi [72310]
damitch247 [72816]
marlowecalleja [62447]
elsawoods [78224]
rachelmitchell_esq [49587]
krstklhs [44261]
candice [153676]


- We have trained 3 models (SVD, KNNBasic and Normal Predictor) and based on RMSE performance, the SVD model performed the best
- We have used grid search for hyperparameters tuning resulting on the best number of items to recommend with a optimum average reviews
- In addition to RMSE we have also calculated the recall and precision for the best K and thresholds
- Our final model outperformed matrix factorization with a RMSE of 1.0422 (vs 15 for the matrix factorization)

Though we have a better and more robust model, that addresses most of the weaknesses from the previous phase, it will be more efficient for popular products with high numbers of reviews. Hence we still need to solve the cold start issue.

The final modelling scenario will leverage the content base approach to adress this issue. 